In [1]:
# Util
import random
import time

# Model
import agentpy as ap
import matplotlib.animation

# Visualization
import matplotlib.pyplot as plt
import IPython

# Server
import client

In [2]:
VALID_MOVES = {
    'RIGHT': (0, 1),
    'DOWN': (1, 0)
}

position = 0
start_time = 0
end_time = 0
wait_time = 0

GREEN = 1
YELLOW = 2
RED = 3
BLUE = 4

COLOR_STRINGS = {
    GREEN: 'GREEN',
    YELLOW: 'YELLOW',
    RED: 'RED',
    BLUE: 'BLUE'
}

class Car(ap.Agent):
    def setup(self):
        self.dir = random.choice(list(VALID_MOVES.keys()))
        self.color = BLUE
        self.type = 'CAR'

    def get_dir(self):
        return VALID_MOVES[self.dir] if random.uniform(0, 1) > 0.1 else (0, 0)

    def get_dir_name(self):
        return self.dir


class TrafficLight(ap.Agent):
    def setup(self):
        global position
        self.color = YELLOW
        self.type = 'TRAFFIC_LIGHT'
        self.pos = position
        position += 1

        
class Interseccion(ap.Model):

    def add_car(self):
        self.new_car = ap.AgentList(self, 1, Car)

        start_positions = {
            'LEFT': (self.v_pos, 0),
            'UP': (0, self.h_pos)
        }

        for car in self.new_car:
            dir_name = car.get_dir_name()

            if dir_name == 'RIGHT':
                position = [start_positions['LEFT']]
            elif dir_name == 'DOWN':
                position = [start_positions['UP']]

        self.grid.add_agents(self.new_car, position)
        self.car_count += 1

        # Add to database
        car = list(self.new_car)[0]
        car_id = car.id
        position = self.grid.positions[car]
        self.client.add_car(car_id, position)
    
    def check_car_horizontal(self):
        started = False
        for neighbor in self.grid.neighbors(self.semaforos[0]):
            semaforo_pos = self.grid.positions[self.semaforos[0]]
            if self.grid.positions[neighbor][1] == semaforo_pos[1] - 1:
                self.semaforos[0].color = GREEN
                self.client.update_traffic_light(self.semaforos[0].id, COLOR_STRINGS[self.semaforos[0].color])
                self.semaforos[1].color = RED
                self.client.update_traffic_light(self.semaforos[1].id, COLOR_STRINGS[self.semaforos[1].color])
                self.duracion_semaforo = self.p.duracion_semaforo
                started = True
                
        return started
                        
    def check_car_vertical(self):
        started = False
        for neighbor in self.grid.neighbors(self.semaforos[1]):
            semaforo_pos = self.grid.positions[self.semaforos[1]]
            if self.grid.positions[neighbor][0] == semaforo_pos[0] - 1:
                self.semaforos[1].color = GREEN
                self.client.update_traffic_light(self.semaforos[1].id, COLOR_STRINGS[self.semaforos[1].color])
                self.semaforos[0].color = RED
                self.client.update_traffic_light(self.semaforos[0].id, COLOR_STRINGS[self.semaforos[0].color])
                self.duracion_semaforo = self.p.duracion_semaforo
                started = True
        
        return started        

    def setup(self):

        self.client = client.Client("http://localhost:8000/")

        # Create streets
        street_length = self.street_length = self.p.size
        self.h_pos = street_length // 2
        self.v_pos = street_length // 2

        # Create lights
        
        self.semaforos = ap.AgentList(self, 2, TrafficLight)
        self.status_semaforos = False
        posiciones_semaforos = [(self.v_pos, self.h_pos-1), (self.h_pos-1, self.v_pos)]
        nombres_semaforos = ['A', 'B']
            
            # semaforos[0]: horizontal
            # semaforos[1]: vertical

        i = 0
        for semaforo in self.semaforos:
            self.client.add_traffic_light(semaforo.id, COLOR_STRINGS[semaforo.color], nombres_semaforos[i])
            i += 1

        # Create grid (calles)
        self.grid = ap.Grid(self, (self.p.size, self.p.size), track_empty=True)
        self.grid.add_agents(self.semaforos, posiciones_semaforos)

        # Counters
        self.step_count = 0
        self.car_count = 0
        #self.duracion_semaforo = self.p.duracion_semaforo
    
    global start_time
    start_time = time.time()

    def step(self):

        global wait_time

        if self.car_count < self.p.n_cars:
            self.add_car()

        if self.semaforos[0].color == YELLOW and self.semaforos[1].color == YELLOW:
            started = self.check_car_horizontal()

            if not started:
                started = self.check_car_vertical()
                
        elif self.semaforos[0].color == GREEN:
            self.duracion_semaforo -= 1
            wait_time += 1
            if self.duracion_semaforo == 0:
                if not self.check_car_vertical():
                    self.semaforos[0].color = YELLOW
                    self.client.update_traffic_light(self.semaforos[0].id, COLOR_STRINGS[self.semaforos[0].color])
                    self.semaforos[1].color = YELLOW
                    self.client.update_traffic_light(self.semaforos[1].id, COLOR_STRINGS[self.semaforos[1].color])  
                    
        elif self.semaforos[1].color == GREEN:
            self.duracion_semaforo -= 1
            wait_time += 1
            if self.duracion_semaforo == 0:
                if not self.check_car_horizontal():
                    self.semaforos[0].color = YELLOW
                    self.client.update_traffic_light(self.semaforos[0].id, COLOR_STRINGS[self.semaforos[0].color])
                    self.semaforos[1].color = YELLOW
                    self.client.update_traffic_light(self.semaforos[1].id, COLOR_STRINGS[self.semaforos[1].color])

        for agent in list(self.grid.agents):
            agent_pos = self.grid.positions[agent]
            move = True
            if agent.type == 'CAR':
                for neighbor in self.grid.neighbors(agent):
                    if agent.get_dir_name() == 'RIGHT':
                        # si el vecino es el de uno en frente y corresponde a la misma calle (posición vertical)
                        if self.grid.positions[neighbor][1] == agent_pos[1] + 1 and self.grid.positions[neighbor][0] == agent_pos[0]:
                            if (neighbor.type == 'TRAFFIC_LIGHT' and neighbor.color == RED) or neighbor.type == 'CAR':
                                move = False
                                break
                    elif agent.get_dir_name() == 'DOWN':
                        if self.grid.positions[neighbor][0] == agent_pos[0] + 1 and self.grid.positions[neighbor][1] == agent_pos[1]:
                            if (neighbor.type == 'TRAFFIC_LIGHT' and neighbor.color == RED) or neighbor.type == 'CAR':
                                move = False
                                break
                if move:
                    self.grid.move_by(agent, agent.get_dir())
                    new_pos = self.grid.positions[agent]
                    self.client.update_car(agent.id, new_pos)

                if (self.grid.positions[agent] == (self.v_pos, self.p.size - 1) or self.grid.positions[agent] == (
                self.p.size- 1, self.h_pos)):
                    self.grid.remove_agents(agent)
                    self.client.delete_car(agent.id)

        if self.p.time == self.step_count:
            self.stop()

        # upload changes to cloud
        self.client.commit()

        self.step_count += 1

        time.sleep(self.p.step_dur)

    def end(self):
        global end_time
        end_time = time.time()

        self.report('Simulation running time',
                     end_time - start_time )

        self.report('Tiempo de espera',
                     wait_time )

        for agent in self.grid.agents:
            self.client.delete_car(agent.id)
            self.client.delete_traffic_light(agent.id)

parameters = {
    'size': 20,
    'n_cars': 8,
    'time': 10000,
    'step_dur': 0.1,
    'duracion_semaforo': 5
}

if __name__ == "__main__":
    model = Interseccion(parameters)
    model.run()


Completed: 1 steps
Completed: 2 steps
Completed: 3 steps
Completed: 4 steps
Completed: 5 steps
Completed: 6 steps
Completed: 7 steps
Completed: 8 steps
Completed: 9 steps
Completed: 10 steps
Completed: 11 steps
Completed: 12 steps
Completed: 13 steps
Completed: 14 steps
Completed: 15 steps
Completed: 16 steps
Completed: 17 steps
Completed: 18 steps
Completed: 19 steps
Completed: 20 steps

ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000231F014CF40>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))